#### Load libraries

In [ ]:
library(SCENIC)
library(Seurat)
library(GENIE3)
library(doParallel)
library(doRNG)

#### Extracting gene expression matrices

Load seurat objects

In [ ]:
VTA.integrated<-readRDS(file="./VTA_integrated.rds")
VTA_YFP <- readRDS(file = "./VTA_YFP.rds")
VTA_LH <- readRDS(file = "./VTA_LH.rds")
VTA_PFC <- readRDS(file = "./VTA_PFC.rds")
VTA_NAc <- readRDS(file = "./VTA_NAc.rds")

Assigning identities and grabbing cell class barcodes

In [ ]:
new.ident <- c("Glu","CoEx","Gaba","DA","DA","Gaba","Glu","Glu","Glu","DA","Glu","Glu","Glu","CoEx","CoEx","Gaba","Gaba")
names(x = new.ident) <- levels(x =VTA.integrated)
VTA.integrated<- RenameIdents(object =VTA.integrated, new.ident)
for (i in 1:length(new.ident)){
  assign(paste(new.ident[i],"_barcode",sep=""),colnames(VTA.integrated@assays$RNA@data[,which(Idents(object=VTA.integrated) %in% new.ident[i])]))# this gives all barcodes in cluster
  assign(paste(new.ident[i],"_barcode_VTA_YFP",sep=""),intersect(colnames(VTA_YFP@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
  assign(paste(new.ident[i],"_barcode_VTA_LH",sep=""),intersect(colnames(VTA_LH@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
  assign(paste(new.ident[i],"_barcode_VTA_NAc",sep=""),intersect(colnames(VTA_NAc@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
  assign(paste(new.ident[i],"_barcode_VTA_PFC",sep=""),intersect(colnames(VTA_PFC@assays$RNA@data),eval(parse(text = paste(new.ident[i],"_barcode",sep="")))))
}
barcode_names <- unique(paste(Idents(VTA.integrated),"_barcode",sep=""))
celltype<-vector()
for (i in 1:dim(VTA.integrated@meta.data)[1]){
  celltype[i]<-toString(new.ident[VTA.integrated@meta.data$integrated_snn_res.0.5[i]])
}
VTA.integrated@meta.data$celltype<-celltype

Sample code to fetch count data for a particular group

In [ ]:
DefaultAssay(VTA.integrated) <- 'RNA'
VTA_DA_YFP <- subset(VTA.integrated, cells=DA_barcode_VTA_YFP)
exprMat <- as.matrix(GetAssayData(VTA_DA_YFP, layer = 'counts'))
saveRDS(exprMat, 'VTA_DA_YFP_mat.rds')

In [ ]:
DefaultAssay(VTA.integrated) <- 'RNA'
VTA_DA_LH <- subset(VTA.integrated, cells=DA_barcode_VTA_LH)
exprMat <- as.matrix(GetAssayData(VTA_DA_LH, layer = 'counts'))
saveRDS(exprMat, 'VTA_DA_LH_mat.rds')

In [ ]:
DefaultAssay(VTA.integrated) <- 'RNA'
VTA_DA_PFC <- subset(VTA.integrated, cells=DA_barcode_VTA_PFC)
exprMat <- as.matrix(GetAssayData(VTA_DA_PFC, layer = 'counts'))
saveRDS(exprMat, 'VTA_DA_PFC_mat.rds')

In [ ]:
DefaultAssay(VTA.integrated) <- 'RNA'
VTA_DA_NAc <- subset(VTA.integrated, cells=DA_barcode_VTA_NAc)
exprMat <- as.matrix(GetAssayData(VTA_DA_NAc, layer = 'counts'))
saveRDS(exprMat, 'VTA_DA_NAc_mat.rds')

Read in matrix objects and combine into one dataframe

In [ ]:
YFP <- readRDS('VTA_DA_YFP_mat.rds')
LH <- readRDS('VTA_DA_LH_mat.rds')
PFC <- readRDS('VTA_DA_PFC_mat.rds')
NAc <- readRDS('VTA_DA_NAc_mat.rds')
exprMat <- cbind(YFP, LH, PFC, NAc)

#### Running SCENIC

Set working directory

In [ ]:
setwd("./DA_all_scenic/")

http://htmlpreview.github.io/?https://github.com/aertslab/SCENIC/blob/master/inst/doc/SCENIC_Running.html

Initializing SCENIC

In [ ]:
org <- "mgi" 
dbDir <- "./cisTarget_databases" 
myDatasetTitle <- "VTA SCENIC"
data(defaultDbNames)
dbs <- defaultDbNames[[org]]
scenicOptions <- initializeScenic(org=org, dbDir=dbDir, dbs=dbs, datasetTitle=myDatasetTitle, nCores=1) 
saveRDS(scenicOptions, file="int/scenicOptions.Rds") 

Generate co-expression network

In [ ]:
genesKept <- geneFiltering(exprMat, scenicOptions=scenicOptions,
                           minCountsPerGene=3*.01*ncol(exprMat),
                           minSamples=ncol(exprMat)*.01)

exprMat_filtered <- exprMat[genesKept, ]
runCorrelation(exprMat_filtered, scenicOptions)
exprMat_filtered_log <- log2(exprMat_filtered+1) 
runGenie3(exprMat_filtered_log, scenicOptions)

Build and score GRN

In [ ]:
exprMat_log <- log2(exprMat+1)
scenicOptions <- runSCENIC_1_coexNetwork2modules(scenicOptions)
scenicOptions <- runSCENIC_2_createRegulons(scenicOptions) 
scenicOptions <- runSCENIC_3_scoreCells(scenicOptions, exprMat_log)

Calculate AUC & RSS

In [ ]:
meta.string <- regulonAUC@colData@rownames
meta <- gsub(".*_","",meta.string)
regulonAUC <- loadInt(scenicOptions, "aucell_regulonAUC")
rss <- calcRSS(AUC=getAUC(regulonAUC), cellAnnotation=meta)
rssPlot <- plotRSS(rss)